In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,roc_curve, roc_auc_score
import warnings

In [2]:
df = pd.read_csv('../65 Years of Weather Data Bangladesh (1948 - 2013).csv')
df.head()

,No,Station_Names,YEAR,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period
0,0,Barisal,1949,1,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1949.01
1,1,Barisal,1950,1,30.0,14.1,0.0,77.0,0.453704,0.8,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1950.01
2,2,Barisal,1951,1,28.2,12.3,0.0,77.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1951.01
3,3,Barisal,1952,1,26.6,12.3,2.0,77.0,0.453704,1.0,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1952.01
4,4,Barisal,1953,1,30.0,13.3,10.0,75.0,0.453704,1.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1953.01


In [3]:

# Players to be selected
li = ["Chittagong (City-Ambagan)","Chittagong (IAP-Patenga)","Sandwip","Sitakunda","Kutubdia","Cox's Bazar","Teknaf","Sylhet","Srimangal","Rangamati"]
 
df_new=df[df.Station_Names.isin(li)]
df_new

,No,Station_Names,YEAR,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period
2748,2784,Chittagong (City-Ambagan),2009,1,31.566667,13.7,0.000000,70.333333,3.033333,0.1,6.411131,41977,0.0,0.0,22.35,91.8166,0,2009.01
2749,2748,Chittagong (City-Ambagan),2011,1,32.400000,13.5,0.000000,67.000000,2.800000,0.9,6.411131,41977,0.0,0.0,22.35,91.8166,0,2011.01
2750,2749,Chittagong (City-Ambagan),2012,1,30.700000,14.9,0.000000,75.000000,3.300000,1.5,6.411131,41977,0.0,0.0,22.35,91.8166,0,2012.01
2751,2750,Chittagong (City-Ambagan),2013,1,31.600000,12.7,0.000000,69.000000,3.000000,0.3,6.411131,41977,0.0,0.0,22.35,91.8166,0,2013.01
2752,2785,Chittagong (City-Ambagan),2009,2,34.833333,16.7,0.333333,62.333333,3.166667,0.7,6.411131,41977,0.0,0.0,22.35,91.8166,0,2009.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21115,21115,Teknaf,2009,12,30.300000,16.5,0.000000,72.000000,2.800000,0.9,8.700000,41998,734765.4,308914.1,20.87,92.2600,4,2009.12
21116,21116,Teknaf,2010,12,31.700000,16.7,193.000000,79.000000,2.400000,1.4,5.500000,41998,734765.4,308914.1,20.87,92.2600,4,2010.12
21117,21117,Teknaf,2011,12,31.500000,16.4,0.000000,73.000000,0.000000,1.5,7.400000,41998,734765.4,308914.1,20.87,92.2600,4,2011.12
21118,21118,Teknaf,2012,12,30.000000,15.8,0.000000,70.000000,1.800000,0.2,9.000000,41998,734765.4,308914.1,20.87,92.2600,4,2012.12


In [4]:
df_new=df_new.drop(labels=["No","Station_Names","YEAR","Month","ALT","Station_Number"], axis=1)
df_new.head()

,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,X_COR,Y_COR,LATITUDE,LONGITUDE,Period
2748,31.566667,13.7,0.000000,70.333333,3.033333,0.1,6.411131,0.0,0.0,22.35,91.8166,2009.01
2749,32.400000,13.5,0.000000,67.000000,2.800000,0.9,6.411131,0.0,0.0,22.35,91.8166,2011.01
2750,30.700000,14.9,0.000000,75.000000,3.300000,1.5,6.411131,0.0,0.0,22.35,91.8166,2012.01
2751,31.600000,12.7,0.000000,69.000000,3.000000,0.3,6.411131,0.0,0.0,22.35,91.8166,2013.01
2752,34.833333,16.7,0.333333,62.333333,3.166667,0.7,6.411131,0.0,0.0,22.35,91.8166,2009.02


In [5]:
z = df_new["Wind_Speed"]

df_new = df_new.drop(labels=["Wind_Speed","X_COR","Y_COR","Bright_Sunshine"], axis=1)
df_new['Wind_Speed'] = z
df_new.head()

,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Cloud_Coverage,LATITUDE,LONGITUDE,Period,Wind_Speed
2748,31.566667,13.7,0.000000,70.333333,0.1,22.35,91.8166,2009.01,3.033333
2749,32.400000,13.5,0.000000,67.000000,0.9,22.35,91.8166,2011.01,2.800000
2750,30.700000,14.9,0.000000,75.000000,1.5,22.35,91.8166,2012.01,3.300000
2751,31.600000,12.7,0.000000,69.000000,0.3,22.35,91.8166,2013.01,3.000000
2752,34.833333,16.7,0.333333,62.333333,0.7,22.35,91.8166,2009.02,3.166667


In [6]:
X = df_new.iloc[:,:-1]
Y = df_new.iloc[:,-1]

In [7]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2)

In [8]:
# Standard Scaler

from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
scaler = sc.fit(trainX)
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)

In [9]:
from xgboost import XGBRegressor

RegModel=XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=500, objective='reg:linear', booster='gbtree')
XGB=RegModel.fit(trainX_scaled,trainY)
prediction=XGB.predict(testX_scaled)

[20:19:31] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [10]:
df_temp = pd.DataFrame({'Actual': testY, 'Predicted': prediction})
df_temp.head()

,Actual,Predicted
11621,1.9,1.812539
20171,1.5,1.246092
19692,0.9,0.834747
19136,0.0,0.265621
20959,1.2,0.717564


In [11]:
#Measuring Goodness of fit in Training data
from sklearn import metrics
print('R2 Value:',metrics.r2_score(testY, prediction))

print('Mean Absolute Error:', metrics.mean_absolute_error(testY, prediction))  
print('Mean Squared Error:', metrics.mean_squared_error(testY, prediction))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(testY, prediction)))

R2 Value: 0.779844298530408
Mean Absolute Error: 0.39593542630502276
Mean Squared Error: 0.3073559736206242
Root Mean Squared Error: 0.5543969458976341
